In [0]:

from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
import keras.backend as K
import matplotlib.pyplot as plt
import sys
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

## Baseline W-DCGAN

In [0]:
img_shape = (64,64,3)
latent_dim = 100

n_critic = 5
clip_value = 0.01
optimizer = RMSprop(lr=0.00005)

### Critic

In [0]:
def discriminator():
  model = Sequential()

  model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(1))

  model.summary()
  
  img = Input(shape=img_shape)
  valid = model(img)
  
  
  return Model(img, valid)
 
  

In [10]:
discriminator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 17, 17, 32)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 17, 17, 32)        128       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 17, 17, 32)        0         
__________

### Generator

In [0]:
def generator():
  model = Sequential()

  model.add(Dense(4096, activation="relu", input_dim=latent_dim))
  model.add(Reshape((4, 4, 256)))
  
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  
  model.add(UpSampling2D())
  model.add(Conv2D(32, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  
  model.add(UpSampling2D())
  model.add(Conv2D(3, kernel_size=3, padding="same")) # 3 rgb channels
  model.add(Activation("tanh"))

  model.summary()
  
  noise = Input(shape=(latent_dim,))
  img = model(noise)
  return Model(noise, img)

In [32]:
generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 4096)              413696    
_________________________________________________________________
reshape_10 (Reshape)         (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_25 (UpSampling (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 8, 8, 128)         295040    
_________________________________________________________________
batch_normalization_29 (Batc (None, 8, 8, 128)         512       
_________________________________________________________________
activation_31 (Activation)   (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_26 (UpSampling (None, 16, 16, 128)       0         
__________

In [0]:
def wloss(y_true, y_pred):
  return K.mean(y_true*y_pred)

In [37]:
d = discriminator()
g = generator()


d.compile(loss=wloss, optimizer=optimizer, metrics=["accuracy"])

z = Input(shape=(latent_dim,))
img = g(z)

d.trainable = False
valid = d(img)

GAN = Model(z, valid)
GAN.compile(loss=wloss, optimizer=optimizer, metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 17, 17, 32)        0         
_________________________________________________________________
batch_normalization_38 (Batc (None, 17, 17, 32)        128       
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 17, 17, 32)        0         
__________

In [0]:
def sample_images(epoch):
  r,c = 5,5
  noise = np.random.normal(0,1, (r*c, latent_dim))
  gen_imgs = g.predict(noise)
  
  gen_imgs = 0.5 * gen_imgs + 0.5
  
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("gan_%d.png" % epoch)
  plt.close()
  

In [0]:
from keras.utils import plot_model
plot_model(GAN, show_shapes=True, show_layer_names=True)

In [0]:
def train(epochs=500, batch_size=128, sample_interval=50):
  
  valid = -np.ones((batch_size, 1)) # negative since using tanh
  fake = np.ones((batch_size, 1))
  generator = g
  discriminator = d
  
 
  datagen_dfc = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)
  X_train_gen = datagen_dfc.flow_from_directory(train_path,
        target_size=(64,64), batch_size=batch_size,
        class_mode=None, color_mode="rgb")
  for epoch in range(epochs):
    print(epoch)
    for _ in range(n_critic):
    
      # Train discriminator
      
      
      imgs = next(X_train_gen)
      
      noise = np.random.normal(0,1, (batch_size, latent_dim))
      
      # generates new image
      gen_imgs = generator.predict(noise)
      
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(imgs, fake)
      d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
      
      for l in discriminator.layers:
        weights = l.get_weights()
        weights = [np.clip(w, -clip_value, clip_value) for w in weights]
        l.set_weights(weights)
    # train generator
    g_loss = GAN.train_on_batch(noise, valid)
    print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))
    if epoch % sample_interval == 0:
      sample_images(epoch)
   
  
  
  

In [41]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [47]:
train_path = "/content/drive/My Drive/train/"
test_path = "/content/drive/My Drive/test/"


Found 17441 images belonging to 1 classes.
Found 867 images belonging to 1 classes.


In [51]:
train(epochs=4000)

Found 17441 images belonging to 1 classes.
0


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.999848] [G loss: 1.000347]
1
1 [D loss: 0.999855] [G loss: 1.000358]
2
2 [D loss: 0.999866] [G loss: 1.000369]
3


KeyboardInterrupt: ignored